# HW2 - Problem 1

In [0]:
import numpy as np
from tensorflow import keras


In [0]:
class Graph:
  """
  Turns a numpy image into a graph
  Allows for finding connected components
  """
  # Choose an arbitrary value to be what we consider "on"

  def pixel_round(self, num):
    return int(num >= 100)
  # init function to declare class variables 
  def __init__(self, image):
    # First, create the graph from the image 
    self.V = len(image) * len(image[0])
    self.adj = []

    v = 0
    for i in range(len(image)):
      for j in range(len(image[i])):
        new_ad = []
        if j > 0 and (self.pixel_round(image[i, j]) == self.pixel_round(image[i, j - 1])):
          new_ad.append(v - 1)
        if j < len(image[i]) - 1 and (self.pixel_round(image[i, j]) == self.pixel_round(image[i, j + 1])):
          new_ad.append(v + 1)
        if i > 0 and (self.pixel_round(image[i, j]) == self.pixel_round(image[i - 1, j])):
          new_ad.append(v - len(image[i]))
        if i < len(image) - 1 and (self.pixel_round(image[i, j]) == self.pixel_round(image[i + 1, j])):
          new_ad.append(v + len(image[i]))
        
        # Apend the new adjacency list
        self.adj.append(new_ad)
        v += 1
  
  def boundedDFS(self, temp, v, visited): 
    """
    DFS until you've encountered either a visited node
    """
  
    # Visit the current v
    visited[v] = True
  
    # store it
    temp.append(v) 
  
    # repeat for everything adjacent to v
    for i in self.adj[v]: 
      if visited[i] == False: 
        # Update the list 
        temp = self.boundedDFS(temp, i, visited) 
    return temp 
  
  # method to add an undirected edge 
  def addEdge(self, v, w): 
    self.adj[v].append(w) 
    self.adj[w].append(v) 
        
  def connectedComponents(self):
    """
    Use DFS to find all connected components
    """
    visited = [] 
    cc = [] 
    for i in range(self.V): 
      visited.append(False) 
    for v in range(self.V): 
      if visited[v] == False: 
        temp = [] 
        new_cc = self.boundedDFS(temp, v, visited) 
        if len(new_cc) > 1:
          cc.append(new_cc)
    return cc 


Our stratgy to build our handcrafted features is as follows:
We first turn the image into a graph, then use DFS to find all connected components
Once we have the number of said components, we find the component corresponding to the image itself and find the width and height by comparing its maximum and minimum elements

In [0]:
def features(image):
  """
  Given an mnist image, returns the number of connected componants, the height,
  and the width of that image
  """
  image = np.array(image)
  g = Graph(image)
  cc = g.connectedComponents()
    
  # Search for the connected component that actually makes up the ones 
    
  for comp in cc:
    if image[comp[0] // 28, comp[0] % 28] != 0:
      true_num = np.array(comp)
      break

  # Get the horizontal position of pixals
  hor_pos = np.mod(true_num, 28)
  
  # simply find the pixel furthest to the left, and the one furthest to the right
  left = np.min(hor_pos)
  right = np.max(hor_pos)
  width = right - left
    
  # Get the vertical position of pixals
  vert_pos = true_num // 28
    
  ## Same as before, just vertical
  top = np.min(vert_pos)
  bottom = np.max(vert_pos)
  height = bottom - top
    
  return np.array([len(cc), height / 10, width / 10])


In [0]:
mnist = keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [0]:
train_images = np.concatenate([features(d) for d in train_images])
test_images = np.concatenate([features(d) for d in test_images])

In [0]:
train_labels = keras.utils.to_categorical(train_labels)
test_labels = keras.utils.to_categorical(test_labels)

In [0]:
test_labels

In [1]:
# set up the layers

import tensorflow as tf
from tensorflow import keras
model = keras.Sequential([

    keras.layers.Dense(30, activation=tf.nn.relu, input_shape=(3, )),
    keras.layers.Dense(15, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

# compile the model

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# train the model
epochs = 50
history = model.fit(train_images, 
                      train_labels, 
                      epochs=epochs,  
                      validation_data=(test_images, test_labels))

NameError: name 'train_images' is not defined